# Lab | Variational Autoencoder (VAE) on Fashion-MNIST Dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, datasets
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
# Figure out how to import regularizers and backend
###
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
# Load and preprocess Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape to flatten the images
x_train = x_train.reshape(x_train.shape[0], 28 * 28)
x_test = x_test.reshape(x_test.shape[0], 28 * 28)

print(f"Training data shape: {x_train.shape}")
print(f"Test data shape: {x_test.shape}")

# Fashion-MNIST class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Display some sample images
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x_train[i].reshape(28, 28), cmap='gray')
    plt.title(class_names[y_train[i]])
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Sampling layer for the VAE
#TODO: YOUR CODE HERE

In [ ]:
# Build the encoder
latent_dim = 2  # 2D latent space for easy visualization
original_dim = 28 * 28
#TODO: YOUR CODE HERE

In [ ]:
# Build the decoder


In [ ]:
# Build the VAE model
#TODO: YOUR CODE HERE

In [ ]:
# Train the VAE
#TODO: YOUR CODE HERE

In [ ]:
# Plot training history
#TODO: YOUR CODE HERE

In [ ]:
# Visualize the latent space
#TODO: YOUR CODE HERE

In [ ]:
#####################################################
# Generate new images by sampling from latent space #
#####################################################

### YOUR CODE HERE


In [ ]:
#############################################
# Compare original vs reconstructed images  #
#############################################


### YOUR CODE HERE